In [6]:
#!pip install langchain==0.0.163
#!pip install pygpt4all==1.1.0
#!pip install transformers
#!pip install datasets
#!pip install chromadb
#!pip install tiktoken


(tensorflow) C:\Users\gsdav\Documents\Data\PythonPrograms\JupyterNotebook\LLM>conda.bat activate langchain 


In [1]:
!conda info


     active environment : langchain
    active env location : C:\Users\gsdav\Anaconda3\envs\langchain
            shell level : 2
       user config file : C:\Users\gsdav\.condarc
 populated config files : C:\Users\gsdav\.condarc
          conda version : 4.8.5
    conda-build version : 3.18.11
         python version : 3.7.6.final.0
       virtual packages : 
       base environment : C:\Users\gsdav\Anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/noarch
          package cache : C:\Users\gsdav\Anaconda3\pkgs
                          C:\Users\gsdav\.conda\pkgs
                          C:\Users\gsdav\AppData\Local

In [2]:
import pandas as pd
from datasets import load_dataset

# Download the medical_dialog dataset from Hugging Face
dataset = load_dataset('medical_dialog', 'processed.en')

# Convert the dataset to a pandas dataframe
df = pd.DataFrame(dataset['train'])

# Print the first 5 rows of the dataframe
df.head()

C:\Users\gsdav\Anaconda3\envs\langchain\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for medical_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/medical_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/482 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61 [00:00<?, ? examples/s]

,description,utterances
0,throat a bit sore and want to get a good imune...,[patient: throat a bit sore and want to get a ...
1,"hey there i have had cold ""symptoms"" for over ...","[patient: hey there i have had cold ""symptoms""..."
2,i have a tight and painful chest with a dry co...,[patient: i have a tight and painful chest wit...
3,what will happen after the incubation period f...,[patient: what will happen after the incubatio...
4,suggest treatment for pneumonia,[patient: just found out i was pregnant. yeste...


In [3]:
dialog = []
# make each sentence on a seperate row
patient, doctor = zip(*df['utterances'])
for i in range(len(patient)):
  dialog.append(patient[i])
  dialog.append(doctor[i])

dialog_df = pd.DataFrame({"dialog": dialog})
# save the data to txt file
dialog_df.to_csv('data.txt', sep=' ', index=False)

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings

# add the path to the CV as a PDF
loader = TextLoader('data.txt')
# Embed the document and store into chroma DB
index = VectorstoreIndexCreator(embedding= HuggingFaceEmbeddings()).from_loaders([loader])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\gsdav\Anaconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gsdav\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# specify the path to the .bin downloaded file
# local_path = './models/ggml-gpt4all-j-v1.3-groovy.bin'  # replace with your desired local file path
local_path = 'C:/Users/gsdav/AppData/Local/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf'
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, backend='gptj')

In [8]:
# perform similarity search and retrieve the context from our documents
results = index.vectorstore.similarity_search("what is the solution for sore throat", k=4)
# join all context information (top 4) into one string 
context = "\n".join([document.page_content for document in results])
print(f"Retrieving information related to your question...")
print(f"Found this content which is most similar to your question: {context}")

Retrieving information related to your question...
Found this content which is most similar to your question: "doctor: in brief: standard precautions covid-19 is now official name for the illness caused by the newly discovered coronavirus (coronavirus infectious disease - 2019). so far it is extremely rare in the us (2/12/20). until and unless covid-19 becomes common no special precautions are necessary. in any dormitory or group living situation people with respiratory symptoms (colds, flu, etc.) should cover their coughs and wash hands frequently."
"patient: is gargling with listerine effective against corona virus induced sore throat? will it kill the virus? how about with mixture of warm water and salt, will this also kill virus!"
"doctor: gargling. you can't be sure but it may help if you do those things as well as using zinc lozenges at the first sign of any throat discomfort and stay hydrated also. i recommend them. at least it'll do no harm."
"doctor: in brief: may call dr on v

In [9]:
template = """
Please use the following context to answer questions.
Context: {context}
---
Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)

llm_chain = LLMChain(prompt=prompt, llm=llm)
# Print the result
print("Processing the information with gpt4all...\n")
print(llm_chain.run("what is the treatment for sore throat"))

Processing the information with gpt4all...

 The patient has a sore throat and wants to know if gargling with Listerine will help kill the virus. The doctor says that while it may not directly kill the virus, gargling along with using zinc lozenges at the first sign of any throat discomfort and staying hydrated can be helpful.
The patient also mentions having had tonsillitis in the past and experiencing allergic problems every year around this time. They take antihistamines but have used prednisone before to treat these issues. The doctor doesn't explicitly say that it's COVID-19, but suggests getting tested for COVID-19.

So, based on what we know from the conversation, the treatment for sore throat is:
1. Gargling with Listerine
2. Using zinc lozenges at the first sign of any throat discomfort
3. Staying hydrated

Please note that these are general suggestions and not a definitive diagnosis or treatment plan. It's always best to consult a healthcare professional for personalized advi